# Phase II

### Importing Data

In [ ]:
%pip install --upgrade pip

In [ ]:
%pip install matplotlib datasets chess numpy
%pip install --upgrade tensorflow


In [ ]:
from datasets import load_dataset

# Streaming DataSet
full_dataset = load_dataset("Lichess/chess-position-evaluations", split="train", streaming=True)
train_dataset = full_dataset.shuffle(seed=42, buffer_size=10_000).take(9_000_000)
val_dataset = full_dataset.skip(9_000_000).take(1_000_000)

# see few values:
for i, sample in enumerate(train_dataset):
    print(sample)
    if i == 2:
        break

### Utility Functions

In [ ]:
import chess
import numpy as np

def board_to_matrix(board):
    matrix = np.zeros((8, 8, 12), dtype=np.float32)
    piece_map = board.piece_map()

    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1  # 0-indexed
        color_offset = 0 if piece.color == chess.WHITE else 6
        channel = piece_type + color_offset
        matrix[row][col][channel] = 1

    return matrix


In [ ]:
def evaluation_to_score(cp, mate):
    if mate is not None:
        return 20.0 if mate > 0 else -20.0
    return cp / 100.0

### Generation DataSet for our Model to Train on

In [ ]:
def data_generator(dataset, batch_size=64):
    batch_x, batch_y = [], []
    for sample in dataset:
        try:    
            fen = sample["fen"]
            board = chess.Board(fen)
            matrix = board_to_matrix(board)
            score = evaluation_to_score(sample['cp'],sample['mate'])

            batch_x.append(matrix)
            batch_y.append(score/1000) # Normalizing Score Range

            if len(batch_x) == batch_size:
                yield np.array(batch_x), np.array(batch_y)
                batch_x, batch_y = [], []
            
        except Exception as e:
            print("Error:", e)
            continue

### Value Detection Model Architech and Training

🧠 What happens when model.fit() runs?
Here's the full flow:

1. model.fit(data_generator(), steps_per_epoch=train_steps) starts training

2. It asks data_generator() for one batch (64 positions)

3. data_generator() collects 64 samples, processes them, and yields a batch

4. The model:

    Gets the batch

    Runs a forward pass to predict scores

    Computes loss (mean squared error)

    Updates model weights via backpropagation

Steps 2–4 repeat train_steps times (e.g. 1000 times)

At the end, the model is trained and saved.

Basically,
My code picks one row at a time from the dataset, processes it into board matrix + score, and groups it into batches of 64. When a batch is ready, it’s passed into the model. The model trains on (TrainSteps) such batches. No need to download the entire dataset, it streams through all files automatically.

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(8, 8, 12)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='linear', padding='same'),
    tf.keras.layers.Conv2D(128, (3, 3), activation='linear', padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='linear'),
    tf.keras.layers.Dense(1, activation='linear')
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.mse())

train_steps = 40000
val_steps = 500
history = model.fit(
    data_generator(train_dataset),
    steps_per_epoch=train_steps,
    validation_data=data_generator(val_dataset),
    validation_steps=val_steps,
    epochs = 10, 
    verbose=1
)


In [ ]:
import matplotlib.pyplot as plt

print(history.history.keys())

# Plot training vs validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Saving the model
model.save(f"chess_eval_model_on_{train_steps}_Epochs{10}Steps.keras")
